# Querying the narrative concepts - Manual Example

This notebook provides information on how to query the manually constructed KG for each of the narrative concept identified in the paper.

## Package Import, Variables, Helpers

In [1]:
import os
import pandas as pd
from rdflib import Graph
from settings import FOLDER_PATH
from src.kg.queries import *

In [2]:
# Format prefixes
prefixes = [x[7:] for x in PREFIXES.split("\n") if x]
PREFIX_F = {x.split(": ")[1].strip()[1:-1]: x.split(": ")[0] for x in prefixes}

def format_uri(uri):
    """ Replace prefixes by abbreviated versions """
    for prefix, abb in PREFIX_F.items():
        uri = uri.replace(prefix, f"{abb}:")
    return uri

In [3]:
# Init graph: KG + ontology
GRAPH = Graph()
GRAPH.parse(
    os.path.join(FOLDER_PATH, "manual-example", 
                 "dul-coup-18-brumaire-simple-rdf.ttl"),
    format="ttl")
GRAPH.parse(
    "http://www.ontologydesignpatterns.org/ont/dul/DUL.owl",
    format='application/rdf+xml')

f"KG & ontology contain {len(list(GRAPH.triples((None, None, None))))} triples"

'KG & ontology contain 2089 triples'

In [4]:
# Helper when querying graph
def query_graph_var(query):
    """ SPARQL query + automatically retrieve variable names"""
    res = list(GRAPH.query(query))
    var = [var[1:] for var in query.replace("DISTINCT ", "").split(" WHERE")[0] \
        .split("SELECT ")[1].split(" ") if var[1:]]
    return res, var

def build_df(res, var):
    """ Build df from the output of the query """
    data = [{v: format_uri(str(triple[i])) for i, v in enumerate(var)} for triple in res]
    return pd.DataFrame(data, columns=var)

## SPARQL Queries

In the `src/kg/queries.py` Python file, you can check the different SPARQL queries related to the different narrative concepts. In the code block below, replace `QUERY_GOAL` by the query you're interested in, and execute the block. You will see the results displayed in a dataframe format.

In [9]:
res, var = query_graph_var(query=QUERY_GOAL)
df = build_df(res, var)
df

,des,sit,o,node,prop
0,ex:Coup_to_take_power_alone,ex:NapoleonCoup18Brumaire,dbr:Napoleon,ex:PowerUsurper,dul:definesRole
1,ex:Coup_to_take_power_alone,ex:NapoleonCoup18Brumaire,dbr:Coup_of_18_Brumaire,dbr:Coup_d%27état,dul:definesTask
